# 飞桨图像分割套件PaddleSeg #

本章主要介绍如何使用PaddleSeg。建议使用GPU环境运行相关程序，可以在启动环境时，如下图所示选择“高级版”环境即可。
<center><img src="https://ai-studio-static-online.cdn.bcebos.com/14a871411dae4bf0b5a2ed0616e9d07759ec593f2f0c4bacb7d495aded5f511b" width="600px" /></center> 
如果没有算力卡资源可以点击链接申请。   
<center><img src="https://ai-studio-static-online.cdn.bcebos.com/2c20ffcc23b04130bace45f5fbdd3ce5a4fe4b362fa4495093daaf04286f9181" width="600px" /></center> 

## **概述** ##

图像分割通过给出图像中每个像素点的标签，将图像分割成若干带类别标签的区块，可以看作对每个像素进行分类。图像分割是图像处理的重要组成部分，也是难点之一。随着人工智能的发展，图像分割技术已经在交通控制、医疗影像和人脸识别等多个领域获得了广泛的应用。如果用户直接使用Python设计、编写图像分割模型并进行训练，则需要消耗较大的工作量。但是现在告诉您，只需要10行左右的代码和命令就可以完成不同应用领域的图像分割功能！如下图所示，只需要使用飞桨的PaddleSeg图像分割模型库即可这样快速使用图像分割技术。

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/676d93118d2a4b72b25d25b27c6946590787ef3e0e8843ea852765937f8bca1a" style="zoom: 80%" /></center>    


飞桨的PaddleSeg图像分割套件如同一个工具箱一样，涵盖了高精度和轻量级等不同方向的大量高质量分割模型，并提供了多个损失函数和多种数据增强方法等高级功能，用户可以根据使用场景从PaddleSeg中选择出合适的图像分割方案，从而更快捷高效地完成图像分割应用。

PaddleSeg的主要特点包括4点：一是提供了50+的高质量预训练模型。二是提供了模块化的设计，支持模型的深度调优；三是高性能计算和显存优化；四是同时支持配置化驱动和API调用两种应用方式，兼顾易用性和灵活性。

### 高精度模型

PaddleSeg提供50+种高质量预训练的分割模型，用户可以选择的预训练模型如 **表1** 所示。


**表1 预训练模型列表**

|模型\骨干网络|ResNet50|ResNet101|HRNetw18|HRNetw48|
|-|-|-|-|-|
|ANN|✔|✔|||
|BiSeNetv2|-|-|-|-|
|DANet|✔|✔|||
|Deeplabv3|✔|✔|||
|Deeplabv3P|✔|✔|||
|Fast-SCNN|-|-|-|-|
|FCN|||✔|✔|
|GCNet|✔|✔|||
|GSCNN|✔|✔|||
|HarDNet|-|-|-|-|
|OCRNet|||✔|✔|
|PSPNet|✔|✔|||
|U-Net|-|-|-|-|
|U<sup>2</sup>-Net|-|-|-|-|
|Att U-Net|-|-|-|-|
|U-Net++|-|-|-|-|

### 模块化设计

1.	支持15+主流 分割网络 ，结合模块化设计的 数据增强策略 、骨干网络、损失函数 等不同组件，开发者可以基于实际应用场景出发，组装多样化的训练配置，满足不同性能和精度的要求。

2.	提供了不同的损失函数，如Cross Entropy Loss、Dice Loss、BCE Loss等类型，通过选择合适的损失函数，可以强化小目标和不均衡样本场景下的分割精度。如 **图4** 所示。


<center><img src="https://ai-studio-static-online.cdn.bcebos.com/b9760df03b844131b852314e6b1800a9092e4bb422b14decbfbaf8f5834e1437" width="800" ></center>
<br><center>图4：PaddleSeg模块化设计</center></br>

### 高性能
PaddleSeg支持多进程异步I/O、多卡并行训练、评估等加速策略，结合飞桨核心框架的显存优化功能，可大幅度减少分割模型的训练开销，让开发者更低成本、更高效地完成图像分割训练。

### 易用灵活
PaddleSeg提供了配置化驱动和API调用两种应用方式，配置化驱动比较简单、容易上手，API调用支持更加灵活地开发。


下面将按以下几个部分进行介绍：
1. 基本原理：简单介绍DeepLabv3+、U-Net、PSPNet、HRNet和Fast-SCNN五类主流分割模型的实现原理。
2. 动态图API使用教程：以BiSeNetV2和医学视盘分割数据集为例一步一步的教导你如何调用API进行模型、数据集、损失函数、优化器等模块的构建。

## **一、基本原理** ##

对于当前十分流行的实时分割场景，例如短视频领域、直播领域，都需要实时分割算法，这就需要我们的算法足够轻量。那么一个好的分割模型网络是什么样子的呢？  首先我们来看下常见的分割网络是什么样的结构。
一个分割网络的输出是一个$N\times H\times W$的概率图，其中N是一个向量，向量的维数表示为类别的数量，也就是说向量的每个元素对应一个类别，而这个元素的数值是概率值，表示为对应该类别的概率。因为要给出每一个像素类别，所以$H\times W$大小需要与原图保持一致。分割网络可以看作是一个没有全连接层的全卷积网络，常见的网络结构可以分为Encoder和Decoder两个部分：
* Encoder部分特征图的长宽会逐步缩小，也就是需要下采样。  
* Decoder部分特征图的长宽会逐步增大，也就是需要上采样。  

一个好的分割网络需要解决哪些问题？第一个问题就是如何通过较少Encoder层数来获得较大的感受野。首先来看下什么是感受野。输出特征图上每个点的数值，是由输入图片上大小为$k_h\times k_w$的区域的元素与卷积核每个元素相乘再相加得到的，所以输入图像上$k_h\times k_w$区域内每个元素数值的改变，都会影响输出点的像素值。我们将这个区域叫做输出特征图上对应点的感受野。感受野内每个像素数值的变动，都会影响输出点的数值变化。因此感受野也可以表示为特征图上每个点所受图像区域范围影响的大小。感受野的大小是分割网络的特征表示能力的一个重要评价指标，因为我们希望尽可能提高感受野的大小。最直接的方法就是提高Encoder网络的层数，但是这种方法会在扩大感受野的同时，带来一些弊端，例如增加了网络参数规模，导致训练耗时增加。那么有没有更好的方法呢？
  
通过扩大卷积核可触达的区域来增大感受野，扩大卷积所包含的信息范围。如下图所示，该图为空洞卷积作用的示意图示意图。可以看到未使用空洞卷积的网络，一个输出的深蓝色的点可以覆盖5个输入像素点。在使用空洞卷积之后，一个输出点可以覆盖9个输入像素的点。其具体应用示例可以参见下面DeepLabv3+的网络结构。

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/b745baf70f5646ed851c2fa96d4657c768828fac58c14298a073c43507838b58" width="800px" /></center> 

分割网络需要面临的第二个问题是下采样过程中特征的长和宽在不断减少，我们虽然得到了高阶特征，但是维度较小，如果直接作为上采样输入，则有可能丢失一部分信息，怎么解决这个问题呢？这时可以使用跳跃连接功能。通过跳跃连接，使上采样过程不仅使用高阶特征，也可以使用同维度的低阶特征，具体使用示例可以参见基本原理中的U-Net模型介绍。

除了上述两个问题之外，分割网络还需要解决多尺度问题，多尺度问题是指同一个图片中目标有大有小，如何保证不同尺度的目标均能被很好的处理呢？这里可以使用ASPP网络结构，通俗的讲，其实现思想是让不同尺度的目标交给不同感受野的卷积层来处理，大感受野处理大目标，小感受野处理小目标。其具体应用示例可以参见下面DeepLabv3+的网络结构。





当前PaddleSeg拥有20+主流分割网络，下面我们将介绍其中几个常用网络的原理。

### DeepLabv3+
DeepLabv3+是DeepLab系列的最后一篇文章，其前作有DeepLabv1、DeepLabv2和DeepLabv3。在最新作中，作者结合编码器-解码器(encoder-decoder)结构和空间金字塔池化模块(Spatial Pyramid Pooling, SPP)的优点提出新的语义分割网络DeepLabv3+，在 PASCAL VOC 2012和Cityscapes数据集上取得新的state-of-art performance.   
其整体结构如下所示，Encoder的主体是带有空洞卷积(Atrous Convolution)的骨干网络，骨干网络可采用ResNet等常用的分类网络，作者使用了改进的Xception模型作为骨干网络。紧跟其后的空洞空间金字塔池化模块(Atrous Spatial Pyramid Pooling, ASPP)则引入了多尺度信息。相比前作DeepLabv3，DeepLabv3+加入decoder模块，将浅层特征和深层特征进一步融合，优化分割效果，尤其是目标边缘的效果。此外，作者将深度可分离卷积(Depthwise Separable Convolution)应用到ASPP和Decoder模块，提高了语义分割的健壮性和运行速率。

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/356ae8f052b94379a4c4bd92b9402aadb085988bfa9043e1a67bca361c3a3bc4" width="600px" /></center> 
<center><br>图1 DeepLabv3+结构图</center> 
 
具体原理细节请参考[Encoder-Decoder with Atrous Separable Convolution for Semantic Image Segmentation](https://arxiv.org/abs/1802.02611)。

### U-Net  
U-Net [1] 起源于医疗图像分割，具有参数少、计算快、应用性强的特点，对于一般场景适应度很高。U-Net最早于2015年提出，并在ISBI 2015 Cell Tracking Challenge取得了第一。经过发展，目前有多个变形和应用。  
原始U-Net的结构是标准的编码器-解码器结构。如下图所示，左侧可视为一个编码器，右侧可视为一个解码器。编码器由四个子模块组成，每个子模块包含两个卷积层，每个子模块之后又通过max pool进行下采样。编码器整体呈现逐渐缩小的结构，不断减少池化层的空间维度，缩小特征图的分辨率，以捕获上下文信息。  
解码器呈现与编码器对称的扩张结构，逐步修复分割对象的细节和空间维度，实现精准的定位。解码器同样也包含四个子模块，分辨率通过上采样操作依次增大，直到与输入图像的分辨率基本一致。  
该网络还使用了跳跃连接，即解码器每上采样一次，就以拼接的方式将解码器和编码器中对应相同分辨率的特征图进行特征融合，帮助解码器更好地恢复目标的细节。由于网络整体结构类似于大写的英文字母U，故得名U-Net。具体原理细节请参考[U-Net:Convolutional Networks for Biomedical Image Segmentation](https://arxiv.org/abs/1505.04597)。

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/d071a65a7d1d411bbf449ba879ba42e91feb3022d22b40468b9c209398c56baf" width="600px" /></center> 
<center><br>图1 U-Net结构图</center> 

### PSPNet
基于语义分割的场景解析是计算机视觉中一个基础话题。自动驾驶、机器人感知等潜在应用领域都十分看重场景解析。场景解析的难度与场景和标签的多样性密切相关。   
Pyramid Scene Parsing Network（PSPNet）起源于场景解析(Scene Parsing)领域。如图3所示，普通FCN面向复杂场景出现三种误分割现象：
* 关系不匹配。将船误分类成车，显然车一般不会出现在水面上。
* 类别混淆。摩天大厦和建筑物这两个类别相近，误将摩天大厦分类成建筑物。
* 类别不显著。枕头区域较小且纹理与床相近，误将枕头分类成床。  


<center><img src="https://ai-studio-static-online.cdn.bcebos.com/ad23fd97ac7b4f4dbf048d7a50db44ace47cd2319f1043d3b4b8d9278c61f092" width="600px" /></center> 
<center><br>图2 普通FCN误分割现象</center> 

PSPNet的出发点是在算法中引入更多的上下文信息来解决上述问题。为了融合图像中不同区域的上下文信息，PSPNet使用了特殊设计的全局均值池化操作（Global Average Pooling）和特征融合构造金字塔池化模块 (Pyramid Pooling Module)。PSPNet最终获得了2016年ImageNet场景解析挑战赛的冠军，并在PASCAL VOC 2012和Cityscapes数据集上取得了当时的最佳效果。具体原理细节请参考[Pyramid Scene Parsing Network](https://arxiv.org/abs/1612.01105)。

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/dce6246f892e4e65b43480f17ff9d5c958ea779823d74e7cb06c49861eb33e45" width="600px" /></center> 
<center><br>图3 PSPNet结构图</center>


### HRNet
现有的大多数方法都是从高分辨率到低分辨率网络(high-to-low resolution network)产生的低分辨率表征中恢复高分辨率表征。但是HRNet（HighResolution Net）能够在整个过程中保持高分辨率表示。如图5所示，HRNet以高分辨率子网开始作为第一阶段，逐个添加高到低分辨率子网以形成更多阶段，并且并行连接多分辨率子网。在整个过程中反复交换并行多分辨率子网络中的信息来进行重复的多尺度融合。在像素级分类、区域级分类和图像级分类中，证明了这些方法的有效性。 HRNet具有两个特点：
* 从高分辨率到低分辨率并行连接各子网络。
* 反复交换跨分辨率子网络信息。  
 
这两个特点使HRNet网络能够学习到更丰富的语义信息和细节信息。 HRNet在人体姿态估计、语义分割和目标检测领域都取得了显著的性能提升。具体原理细节请参考[Deep High-Resolution Representation Learning for Visual Recognition](https://arxiv.org/abs/1908.07919)。

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/fcd7f847e9d14f29a79abd7685df12ede2ec9d1a9aa742ddbf6291e3e248b5a7" width="600px" /></center> 
<center><br>图5 HRNet结构图</center>
    
### Fast-SCNN
Fast-SCNN 是一个面向实时的语义分割网络。在双分支的结构基础上，大量使用了深度可分离卷积和逆残差（inverted-residual）模块，并且使用特征融合构造金字塔池化模块 (Pyramid Pooling Module)来融合上下文信息。这使得Fast-SCNN在保持高效的情况下能学习到丰富的细节信息。Fast-SCNN最大的特点是“小快灵”，即该模型在推理计算时仅需要较小的FLOPs，就可以快速推理出一个不错的结果。

具体原理细节请参考[Fast-SCNN: Fast Semantic Segmentation Network](https://arxiv.org/abs/1902.04502)

整个网络结构如下：
<center><img src="https://ai-studio-static-online.cdn.bcebos.com/15806efe8256435985c331122c1c43f5a63f85d71743473f8588b0f18842fa7d" width="600px" /></center> 
<center><br>图6 Fast-SCNN结构图</center>

### BiSeNetv2  

2020年，相关学者基于可实现实时语义分割的双向网络BiSeNet（Bilateral Segmentation Network），建设性地提出了BiSeNetv2，以期望获得更好的语义分割精度。通常来说，语义分割任务既需要低级细节，也需要高级语义。就目前的语义分割模型而言，于精度相比，更多的是追求速度，因此存在着一定缺陷。而BiSeNetv2则是两方兼顾且行之有效的语义分割模型。其整体结构如下所示：  
  
<center><img src="https://ai-studio-static-online.cdn.bcebos.com/7aadcf87bfd745d6a25921fb13177d7217ec2cff91b643a7a62e176b4e70bc7f" width="600px" /></center> 
<center><br>图7 BiSeNetv2结构图</center>   
  
在骨干网络部分，作者设计了两个分支——细节分支（Detail Branch）和语义分支（Semantic Branch）。细节分支用来捕捉低级细节，语义分支用来获取高级语义内容。为了让两个分支的互补作用更强，模型中加入了聚合层（Aggregation Layer），用来融合两个分支的特征表示。同时，作者还引入了增强训练策略（Booster Training Strategy），提高模型的语义分割效果。
与BiSeNet比较，它的架构更为清晰明了、简单易懂，同时，该模型的网络结构、组件设计也有了较大的提升，从而保障细节部分。总而言之，BiSeNetv2保证速度的同时，也能保证精度。具体原理细节请参考[BiSeNet V2: Bilateral Network with Guided Aggregation forReal-time Semantic Segmentation](https://arxiv.org/pdf/2004.02147.pdf)。   

### OCRNet  
FCN（Fully Convolutional Network for Semantic Segmentation）可以对图像进行像素级的分类，解决了语义级别的图像分割问题，因此现有的大多数语义分割方法都基于FCN。但这些方法也有一定缺陷，比如分辨率低、上下文信息缺失和边界错误等。2020年，相关学者为解决语义分割上下文信息缺失难题，建设性地提出OCRNet，即基于物体上下文特征表示（Object Contextual Representation，以下简称OCR）的网络框架。其整体结构如下所示：   
  
<center><img src="https://ai-studio-static-online.cdn.bcebos.com/c2fe98e4ed81462f95fa057e5b112166cce6bcc76b5b455ca491d2dd0c4b5cc0" width="600px" /></center> 
<center><br>图8 OCRNet结构图</center>   
   
实现此OCR方法需要经历三个阶段——首先形成软物体区域（Soft Object Regions），然后计算物体区域表示（Object Region Representations），最后得到物体上下文特征表示和上下文信息增强的特征表示（Augmented Representation）。
与其他语义分割方法相比，OCR方法更加高效准确。因为OCR方法解决的是物体区域分类问题，而非像素分类问题，即OCR方法可以有效地、显式地增强物体信息。从性能和复杂度来说，OCRNet也更为优秀。2020年，“HRNet + OCR + SegFix”版本在2020ECCV Cityscapes 获得了第一名。具体原理细节请参考[Object-Contextual Representations for SemanticSegmentation](https://arxiv.org/pdf/1909.11065.pdf)。

### GSCNN   
色彩、形状和纹理等低级语义信息并不相同，仅用单一网络融合处理，无法突出需要识别的内容，使得识别效果较差。为解决该问题，相关学者提出了用于语义分割的双流CNN结构（Gated Shape CNNs，以下简称GSCNN）。其整体结构如下所示：   
   
<center><img src="https://ai-studio-static-online.cdn.bcebos.com/b6fca1dc0b85485eb7722dfc313b6de71c43cce4410848518eb56eabef5e95a2" width="600px" /></center> 
<center><br>图9 GSCNN结构图</center>   
   
GSCNN包含了两个流——常规流（Regular Stream）和形状流（Shape Stream）。常规流可以是任意的骨干网络结构。形状流通过一系列残差块（residual block）、门控卷积层（Gated Convolutional Layers，GCL）和边界信息监督，从而保留较多的边界信息。从图可知，双流并列进行，而后共同作为融合层的输入。融合模型（Fusion Module）通过ASPP（Atrous Spatial Pyramid Pooling）模块，融合双流信息。双任务正则化（Dual Task Regularizer）的应用，能够提升边界信息的精度，从而让图像分割更准确。   
该网络结构的亮点是引入了“门”概念，控制两个流的交互，使得形状流过滤掉其他杂质，只处理和边界相关信息。GSCNN在Cityspaces数据集合上表现良好，mIoU指标和F-score指标十分亮眼。与其他方法相比，GSCNN是高效的、用于语义分割的结构。具体原理细节请参考[Gated-SCNN: Gated Shape CNNs for Semantic Segmentation](https://arxiv.org/pdf/1907.05740.pdf)。



### 如何选择模型？
用户可以根据使用场景选择合适的模型：
* 如果是图像分割的初学者，则推荐使用U-Net、FCN模型。
* 如果希望以较快的速度完成训练和预测，则推荐使用Fast-SCNN、BiSeNetv2模型。
* 如果希望获得最高的精度，则推荐使用OCRNet、GSCNN或DeepLabv3+模型。


## **二、动态图API使用教程** ##
PaddleSeg动态图提供了配置化驱动和API调用两种应用方式，配置化驱动比较简单、容易上手，API调用支持更加灵活地开发。

下面介绍PaddleSeg的**API调用**方式，以BiSeNetV2和医学视盘分割数据集为例一步一步的教导你如何调用API进行模型、数据集、损失函数、优化器等模块的构建。如果想了解配置化驱动的使用方法，可点击[PaddleSeg入门教程](https://aistudio.baidu.com/aistudio/projectdetail/1049380)。

### 前置条件

在使用PaddleSeg训练图像分割模型之前，用户需要完成如下任务：  
1. 安装[Python3.6或更高版本](https://www.python.org/downloads/)。   
2. 安装飞桨最新版本，具体安装方法请参见[快速安装](https://www.paddlepaddle.org.cn/install/quick)。由于图像分割模型计算开销大，推荐在GPU版本的PaddlePaddle下使用PaddleSeg。  
3. 下载PaddleSeg的代码库。

In [ ]:
!pip install paddleseg

### 模型训练  

#### 1. 构建模型

In [2]:
from paddleseg.models import BiSeNetV2
model = BiSeNetV2(num_classes=2,
                 lambd=0.25,
                 align_corners=False,
                 pretrained=None)


W1018 16:03:59.081741    98 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 12.0, Runtime API Version: 11.2
W1018 16:03:59.086504    98 gpu_resources.cc:91] device: 0, cuDNN Version: 8.2.


#### 2. 构建训练集及数据预处理流程

#### 数据准备
PaddleSeg目前支持CityScapes、ADE20K、Pascal VOC等数据集的加载，在加载数据集时，如若本地不存在对应数据，则会自动触发下载(除Cityscapes数据集).

1）cityscapes数据集包含了50个不同城市的数据，这个数据集合主要是做城市街景分割，拥有5000张（2048 x 1024）城市驾驶场景的高质量像素级注释图像，包含19个类别。其中训练集2975张， 验证集500张和测试集1525张。

2）Pascal VOC 2012是一个世界级的计算机视觉挑战赛，历史悠久 影响力大，有21个类别，其中训练集1464张，验证集1449张。 通常情况下会利用SBD(Semantic Boundaries Dataset)进行扩充，扩充后训练集10582张。

3）ADE20K由MIT发布的可用于场景感知、分割和多物体识别等多种任务的数据集。其涵盖了150个语义类别，包括训练集20210张，验证集2000张。

这几个数据集都可以用来做分割的实验。

**自定义数据集：**

如果您需要接入其他的数据集进行训练，请按照以下步骤准备数据.

1.推荐整理成如下结构

    custom_dataset
        |
        |--images
        |  |--image1.jpg
        |  |--image2.jpg
        |  |--...
        |
        |--labels
        |  |--label1.jpg
        |  |--label2.png
        |  |--...
        |
        |--train.txt
        |
        |--val.txt
        |
        |--test.txt

其中train.txt和val.txt的内容如下所示：

    images/image1.jpg labels/label1.png
    images/image2.jpg labels/label2.png
    ...

2.标注图像的标签从0,1依次取值，不可间隔。若有需要忽略的像素，则按255进行标注。



本章节将使用视盘分割（optic disc segmentation）数据集进行训练，视盘分割是一组眼底医疗分割数据集，包含了267张训练图片、76张验证图片、38张测试图片。

数据集的原图和效果图如下所示。  
<center><img src="https://ai-studio-static-online.cdn.bcebos.com/cfbcf0eb9a544a94961a5a586edcc190ae8543d8612141d08f78efb917bc6fd0" /></center> 

In [3]:
# 构建训练用的数据增强和预处理
import paddleseg.transforms as T
transforms = [
    T.Resize(target_size=(512, 512)),
    T.RandomHorizontalFlip(),
    T.Normalize()
]

# 构建训练集
from paddleseg.datasets import OpticDiscSeg
train_dataset = OpticDiscSeg(
    dataset_root='data/optic_disc_seg',
    transforms=transforms,
    mode='train'
)

Connecting to https://paddleseg.bj.bcebos.com/dataset/optic_disc_seg.zip
[==================================================] 100.00%
Uncompress optic_disc_seg.zip
[==================================================] 100.00%


#### 数据增强
为什么需要做数据增强？虽然我们已知的公开图像数据集数量很丰富，如ImageNet1k数据集包含128W张图片，即使不加其他策略训练，一般也能获得很高的精度。而在大部分实际业务场景中，需要使用自己的数据集进行训练，但是因为标注难度大且成本高，能获取的标注数据有限；另外线上应用中图像场景一般比较复杂，如可能存在图像遮挡、尺寸变化大等问题，这会导致训练和推理效果达不到预期。

这时，通过一些数据增强的方式去扩充训练样本，可以增加训练样本的丰富度，提升模型的泛化能力，并且可根据场景定制化数据增广。

PaddleSeg内置了图像Resize、图像翻转、图像裁剪、颜色空间扰动等10+种数据增强策略，如 **图10** 直观呈现了多种数据增强策略的效果。

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/b6732a6c81a3497f929f02002ab21bf2f3c53e761cd84864b843081304077f60" width="800" ></center>
<br><center>图10：丰富的数据增强策略</center></br>

我们以图像Resize策略为例，看一下具体是采用什么方式变换的。

图像Resize是指将输入图像按照某种规则，将图片重新缩放到某一个尺寸，包括三种方式：

* Resize 将输入图像直接Resize到某一个固定大小。
* ResizeStepScaling 将输入图像按照某一个比例Resize，这个比例以某一个步长在一定范围内随机变动。
* ResizeRangeScaling 将输入图像按照长边变化进行Resize，即图像长边对齐到某一长度，该长度在一定范围内随机变动，短边随同样的比例变化。

想要了解所有的数据增强API，请参见[paddleseg.transforms](https://github.com/PaddlePaddle/PaddleSeg/blob/develop/docs/apis/transforms.md)

#### 3. 构建验证集及数据预处理流程

In [5]:
# 构建验证用的数据增强和预处理
import paddleseg.transforms as T
transforms = [
    T.Resize(target_size=(512, 512)),
    T.Normalize()
]

# 构建验证集
from paddleseg.datasets import OpticDiscSeg
val_dataset = OpticDiscSeg(
    dataset_root='data/optic_disc_seg',
    transforms=transforms,
    mode='val'
)

#### 4. 构建优化器

In [6]:
import paddle
# 设置学习率
base_lr = 0.01
lr = paddle.optimizer.lr.PolynomialDecay(base_lr, power=0.9, decay_steps=1000, end_lr=0)

optimizer = paddle.optimizer.Momentum(lr, parameters=model.parameters(), momentum=0.9, weight_decay=4.0e-5)

#### 5. 构建损失函数
为了适应多路损失，损失函数应构建成包含'types'和'coef'的dict，如下所示。  其中losses['type']表示损失函数类型， losses['coef']为对应的系数。需注意len(losses['types'])应等于len(losses['coef'])。

In [7]:
from paddleseg.models.losses import CrossEntropyLoss
losses = {}
losses['types'] = [CrossEntropyLoss()] * 5
losses['coef'] = [1]* 5

#### 6. 训练

In [7]:
from paddleseg.core import train
train(
    model=model,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    optimizer=optimizer,
    save_dir='output',
    iters=1000,
    batch_size=4,
    save_interval=200,
    log_iters=10,
    num_workers=0,
    losses=losses,
    use_vdl=True)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/nn/layer/norm.py:654: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/math_op_patch.py:278: UserWarning: The dtype of left and right variables are not the same, left dtype is paddle.float32, but right dtype is paddle.int64, the right dtype will convert to paddle.float32
  format(lhs_dtype, rhs_dtype, lhs_dtype))


2023-10-18 09:50:52 [INFO]	[TRAIN] epoch: 1, iter: 10/1000, loss: 2.3765, lr: 0.009919, batch_cost: 0.2974, reader_cost: 0.02141, ips: 13.4486 samples/sec | ETA 00:04:54
2023-10-18 09:50:53 [INFO]	[TRAIN] epoch: 1, iter: 20/1000, loss: 0.5929, lr: 0.009829, batch_cost: 0.1040, reader_cost: 0.00021, ips: 38.4660 samples/sec | ETA 00:01:41
2023-10-18 09:50:54 [INFO]	[TRAIN] epoch: 1, iter: 30/1000, loss: 0.3859, lr: 0.009739, batch_cost: 0.1028, reader_cost: 0.00023, ips: 38.9029 samples/sec | ETA 00:01:39
2023-10-18 09:50:55 [INFO]	[TRAIN] epoch: 1, iter: 40/1000, loss: 0.3251, lr: 0.009648, batch_cost: 0.1040, reader_cost: 0.00020, ips: 38.4786 samples/sec | ETA 00:01:39
2023-10-18 09:50:56 [INFO]	[TRAIN] epoch: 1, iter: 50/1000, loss: 0.2865, lr: 0.009558, batch_cost: 0.1037, reader_cost: 0.00022, ips: 38.5706 samples/sec | ETA 00:01:38
2023-10-18 09:50:57 [INFO]	[TRAIN] epoch: 1, iter: 60/1000, loss: 0.2813, lr: 0.009467, batch_cost: 0.1043, reader_cost: 0.00021, ips: 38.3344 samples

### 模型评估

#### 1. 构建模型

In [8]:
from paddleseg.models import BiSeNetV2
model = BiSeNetV2(num_classes=2,
                 lambd=0.25,
                 align_corners=False,
                 pretrained=None)

#### 2. 加载模型参数

In [9]:
model_path = 'output/best_model/model.pdparams'
if model_path:
    para_state_dict = paddle.load(model_path)
    model.set_dict(para_state_dict)
    print('Loaded trained params of model successfully')
else: 
    raise ValueError('The model_path is wrong: {}'.format(model_path))

Loaded trained params of model successfully


#### 3. 构建验证集

In [10]:
# 构建验证用的transforms
import paddleseg.transforms as T
transforms = [
    T.Resize(target_size=(512, 512)),
    T.Normalize()
]

# 构建验证集
from paddleseg.datasets import OpticDiscSeg
val_dataset = OpticDiscSeg(
    dataset_root='data/optic_disc_seg',
    transforms=transforms,
    mode='val'
)

#### 4. 评估

In [11]:
from paddleseg.core import evaluate
evaluate(
        model,
        val_dataset)

2023-04-23 09:19:32 [INFO]	Start evaluating (total_samples: 76, total_iters: 76)...


 3/76 [>.............................] - ETA: 3s - batch_cost: 0.0486 - reader cost: 0.005476/76 [==============================] - 3s 35ms/step - batch_cost: 0.0344 - reader cost: 3.9772e-0


2023-04-23 09:19:35 [INFO]	[EVAL] #Images: 76 mIoU: 0.8513 Acc: 0.9940 Kappa: 0.8265 Dice: 0.9133
2023-04-23 09:19:35 [INFO]	[EVAL] Class IoU: 
[0.9939 0.7088]
2023-04-23 09:19:35 [INFO]	[EVAL] Class Precision: 
[0.9962 0.8673]
2023-04-23 09:19:35 [INFO]	[EVAL] Class Recall: 
[0.9977 0.795 ]


(0.8513400256972725,
 0.9940019275129547,
 array([0.99391307, 0.70876698]),
 array([0.99617168, 0.86726204]),
 0.8265185130079017)

#### 5. 多尺度+翻转评估

In [12]:
evaluate(
        model,
        val_dataset,
        aug_eval=True,
        scales=[0.75, 1.0, 1.25],
        flip_horizontal=True)

2023-04-23 09:19:54 [INFO]	Start evaluating (total_samples: 76, total_iters: 76)...


76/76 [==============================] - 15s 194ms/step - batch_cost: 0.1921 - reader cost: 4.4186e-


2023-04-23 09:20:09 [INFO]	[EVAL] #Images: 76 mIoU: 0.8635 Acc: 0.9946 Kappa: 0.8428 Dice: 0.9214
2023-04-23 09:20:09 [INFO]	[EVAL] Class IoU: 
[0.9946 0.7324]
2023-04-23 09:20:09 [INFO]	[EVAL] Class Precision: 
[0.9963 0.8971]
2023-04-23 09:20:09 [INFO]	[EVAL] Class Recall: 
[0.9983 0.7996]


(0.8634744193402624,
 0.9946356707216317,
 array([0.99455574, 0.7323931 ]),
 array([0.99625874, 0.89708235]),
 0.8428072237549258)

### 效果可视化

#### 1. 构建模型

In [13]:
from paddleseg.models import BiSeNetV2
model = BiSeNetV2(num_classes=2,
                 lambd=0.25,
                 align_corners=False,
                 pretrained=None)

#### 2. 创建transform

In [14]:
import paddleseg.transforms as T
transforms = T.Compose([
    T.Resize(target_size=(512, 512)),
    T.RandomHorizontalFlip(),
    T.Normalize()
])

#### 3. 构建待预测的图像列表

In [15]:
import os
def get_image_list(image_path):
    """Get image list"""
    valid_suffix = [
        '.JPEG', '.jpeg', '.JPG', '.jpg', '.BMP', '.bmp', '.PNG', '.png'
    ]
    image_list = []
    image_dir = None
    if os.path.isfile(image_path):
        if os.path.splitext(image_path)[-1] in valid_suffix:
            image_list.append(image_path)
    elif os.path.isdir(image_path):
        image_dir = image_path
        for root, dirs, files in os.walk(image_path):
            for f in files:
                if os.path.splitext(f)[-1] in valid_suffix:
                    image_list.append(os.path.join(root, f))
    else:
        raise FileNotFoundError(
            '`--image_path` is not found. it should be an image file or a directory including images'
        )

    if len(image_list) == 0:
        raise RuntimeError('There are not image file in `--image_path`')

    return image_list, image_dir
image_path = 'data/optic_disc_seg/JPEGImages/N0010.jpg' # 也可以输入一个包含图像的目录
image_list, image_dir = get_image_list('data/optic_disc_seg/JPEGImages/N0010.jpg')

#### 4. 预测
图片预测结果将会输出到保存路径`save_dir`当中。该路径下将生成2个目录，`pseudo_color_prediction`保存伪彩色预测结果图，可直接查看各个类别的预测效果，`added_prediction`保存伪彩色预测结果和原图的叠加效果图。

In [16]:
from paddleseg.core import predict
predict(
        model,
        model_path='output/best_model/model.pdparams',
        transforms=transforms,
        image_list=image_list,
        image_dir=image_dir,
        save_dir='output/results'
    )

2023-04-23 09:20:27 [INFO]	Loading pretrained model from output/best_model/model.pdparams
2023-04-23 09:20:27 [INFO]	There are 356/356 variables loaded into BiSeNetV2.
2023-04-23 09:20:27 [INFO]	Start to predict...
2023-04-23 09:20:27 [INFO]	Predicted images are saved in output/results/added_prediction and output/results/pseudo_color_prediction .


1/1 [==============================] - 0s 66ms/step


预测效果如下：

* 伪彩色预测结果

![](https://ai-studio-static-online.cdn.bcebos.com/7b7d2a45fd0c4697a00d3193c881fe3a99ebde05f9a14482bf4e283677e24335)

* 叠加效果

![](https://ai-studio-static-online.cdn.bcebos.com/6a4255f584bf4f96917d8374609ec766cea4a01a556c45b683fa2003f1e4a769)


## 相关参考链接

* 想要深入了解PaddleSeg源码，可参考[PaddleSeg 代码解读](https://aistudio.baidu.com/aistudio/projectdetail/1136799)

* 更全面的PaddleSeg介绍可参考[Github链接](https://github.com/PaddlePaddle/PaddleSeg)。